## Library

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action="ignore")

DATA_PATH = "/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/"

## import data

In [2]:
df_train = pd.read_csv(DATA_PATH+"train.csv")
df_test = pd.read_csv(DATA_PATH+"test.csv")

In [3]:
pd.set_option("display.max_colwidth", None) # 텍스트 데이터 생략 없이 전체 표기.
df_train.sample(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
1578676,6053509,0.0,Mindfulness meditation has also helped me a great deal – more than words could say…,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,383373,approved,0,0,0,0,0,0.0,0,4
1503549,5961044,0.0,Good luck making Dak nervous. Not sure if I ever saw him nervous.,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,377637,approved,0,0,0,0,0,0.0,0,4
1706247,6213986,0.0,"This would be the case with any Republican EPA nominee, though. The entire goal of the party for decades has been to undermine the administrative state by selling it off piece by piece, neglecting its responsibilities, and blaming the very concept of governance when their mismanaged agency fails.\n\nAdministrative agencies need to be reformed so that their leaders are dedicated career employees who believe in the agency's mission, not political appointees looking to gut the system they oversee.",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,392468,approved,0,0,0,3,1,0.0,0,4
1466312,5913409,0.0,"Pause,\n\nInsurance companies will likely be refusing to offer flood insurance in any region where hurricanes strike. A category 4 hurricane in 2017 will cause a lot more damage and loss of life than a category 4 storm did 50 years ago. We know that our green house gas emissions are warming the oceans and accelerating water cycles. More intense/disastrous rainfall events are being seen now and they will be even more frequent in future years.",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,375013,approved,1,0,0,7,0,0.0,0,4
153776,430204,0.0,"When the scientific case has been made for expansion time and again, by highly credentialed scientists in Hawaii and around the world, it is perplexing to hear this argument, really more of a red herring at this point. And I find the reference to ""pseudoscience"" especially laughable, as though we were discussing chemtrails or sasquatch here. We are not.\n\nIt's also unfortunate that the author attempts to discredit the testifiers in support of expansion, referencing ""a number of high school, college, and graduate students"" to suggest supporters are some sort of ""JV team,"" without any clout, gravitas, or expertise. For one, I find it personally inspiring to see high school students stand up and speak out, and they should be commended, not denigrated. I'm in my 40s and testifying at any public meeting is still terrifying to some degree for me, so I say good for them. They are why this expansion needs to happen, this is for their future, and that of their descendants. (more...)",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,143522,approved,0,0,0,6,0,0.0,0,4
262314,564323,0.0,"So you think her behavior is OK? Please tell us how you justify any of what she has done with the email issues alone. Leave out all the rest. I myself hold my elected officials to very high standards. I worked for the feds for over 33 years and had a high clearance, mainly because of the computer access I had. I would still be in the clink if I did anything remotely like she has.",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,150032,approved,0,0,0,3,0,0.0,0,4
566435,935621,0.0,thank you,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,165550,approved,0,0,0,8,0,0.0,0,4
47257,299752,0.0,"Bad endorsement by the Register Guard. Gary Malone is the far more qualified candidate, with all of his decades of experiencing working in the power generating industry. His has practical hands-on experience, while Carlson is just a graduate student at the U of O. And her close ties to politics is disturbing. We need commissioners who will do what is best for the citizens and their pocket books, and not a person out to pursue a political agenda.\n\nThis whole obsession with sustainability is what has made EWEB's rates go so sky high. That decision to use that Seneca BIOMASS plant, which produces horrible air pollution, and extremely expensive electricity was insane. We cannot

In [4]:
df_train.shape

(1804874, 45)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 45 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [6]:
def missing_count(df: pd.DataFrame) -> pd.Series:
    """
    각 컬럼별 결측치 개수를 반환하는 함수
    df.count() 기반으로 계산
    """
    n_rows = len(df)
    return n_rows - df.count()
    # df.count()는 non-null만 세기에 isnull.sum()보다 빠름

missing = missing_count(df_train)
print(missing)

id                                           0
target                                       0
comment_text                                 3
severe_toxicity                              0
obscene                                      0
identity_attack                              0
insult                                       0
threat                                       0
asian                                  1399744
atheist                                1399744
bisexual                               1399744
black                                  1399744
buddhist                               1399744
christian                              1399744
female                                 1399744
heterosexual                           1399744
hindu                                  1399744
homosexual_gay_or_lesbian              1399744
intellectual_or_learning_disability    1399744
jewish                                 1399744
latino                                 1399744
male         

In [7]:
# 문자형 unique 확인
df_train.describe(include="object").T

,count,unique,top,freq
comment_text,1804871,1780822,Well said.,184
created_date,1804874,1804362,2015-10-13 18:40:35.757707+00,4
rating,1804874,2,approved,1684758


In [8]:
# 수치형 
df_train.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
id,1804874.0,3738433.79,2445186.76,59848.0,796975.25,5223774.5,5769854.50,6334010.00
target,1804874.0,0.10,0.20,0.0,0.00,0.0,0.17,1.00
severe_toxicity,1804874.0,0.00,0.02,0.0,0.00,0.0,0.00,1.00
obscene,1804874.0,0.01,0.06,0.0,0.00,0.0,0.00,1.00
identity_attack,1804874.0,0.02,0.08,0.0,0.00,0.0,0.00,1.00
insult,1804874.0,0.08,0.18,0.0,0.00,0.0,0.09,1.00
threat,1804874.0,0.01,0.05,0.0,0.00,0.0,0.00,1.00
asian,405130.0,0.01,0.09,0.0,0.00,0.0,0.00,1.00
atheist,405130.0,0.00,0.05,0.0,0.00,0.0,0.00,1.00
bisexual,405130.0,0.00,0.03,0.0,0.00,0.0,0.00,1.00
